In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
import xgboost as xgb
import numpy as np

In [ ]:
TRAINING=False

In [ ]:
def reduce_memory_usage(df):
    
    start_memory = df.memory_usage().sum() / 1024**2
    print(f"Memory usage of dataframe is {start_memory} MB")
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != 'object':
            c_min = df[col].min()
            c_max = df[col].max()
            
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    pass
        else:
            df[col] = df[col].astype('category')
    
    end_memory = df.memory_usage().sum() / 1024**2
    print(f"Memory usage of dataframe after reduction {end_memory} MB")
    print(f"Reduced by {100 * (start_memory - end_memory) / start_memory} % ")
    return df

In [ ]:
train = pd.read_csv('../input/jane-street-market-prediction/train.csv')
train = train.query('date > 85').reset_index(drop = True) 
train = train.astype({c: np.float32 for c in train.select_dtypes(include='float64').columns}) #limit memory use
train.fillna(train.mean(),inplace=True)
train = train.query('weight > 0').reset_index(drop = True)
#train['action'] = (train['resp'] > 0).astype('int')
train['action'] =  (  (train['resp_1'] > 0 ) & (train['resp_2'] > 0 ) & (train['resp_3'] > 0 ) & (train['resp_4'] > 0 ) &  (train['resp'] > 0 )   ).astype('int')
features = [c for c in train.columns if 'feature' in c]

resp_cols = ['resp_1', 'resp_2', 'resp_3', 'resp', 'resp_4']

In [ ]:
# train = reduce_memory_usage(train)


In [ ]:
f_mean = np.mean(train[features[1:]].values,axis=0)

In [ ]:
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args

# https://github.com/getgaurav2/scikit-learn/blob/d4a3af5cc9da3a76f0266932644b884c99724c57/sklearn/model_selection/_split.py#L2243
class GroupTimeSeriesSplit(_BaseKFold):
    """Time Series cross-validator variant with non-overlapping groups.
    Provides train/test indices to split time series data samples
    that are observed at fixed time intervals according to a
    third-party provided group.
    In each split, test indices must be higher than before, and thus shuffling
    in cross validator is inappropriate.
    This cross-validation object is a variation of :class:`KFold`.
    In the kth split, it returns first k folds as train set and the
    (k+1)th fold as test set.
    The same group will not appear in two different folds (the number of
    distinct groups has to be at least equal to the number of folds).
    Note that unlike standard cross-validation methods, successive
    training sets are supersets of those that come before them.
    Read more in the :ref:`User Guide <cross_validation>`.
    Parameters
    ----------
    n_splits : int, default=5
        Number of splits. Must be at least 2.
    max_train_size : int, default=None
        Maximum size for a single training set.
    Examples
    --------
    >>> import numpy as np
    >>> from sklearn.model_selection import GroupTimeSeriesSplit
    >>> groups = np.array(['a', 'a', 'a', 'a', 'a', 'a',\
                           'b', 'b', 'b', 'b', 'b',\
                           'c', 'c', 'c', 'c',\
                           'd', 'd', 'd'])
    >>> gtss = GroupTimeSeriesSplit(n_splits=3)
    >>> for train_idx, test_idx in gtss.split(groups, groups=groups):
    ...     print("TRAIN:", train_idx, "TEST:", test_idx)
    ...     print("TRAIN GROUP:", groups[train_idx],\
                  "TEST GROUP:", groups[test_idx])
    TRAIN: [0, 1, 2, 3, 4, 5] TEST: [6, 7, 8, 9, 10]
    TRAIN GROUP: ['a' 'a' 'a' 'a' 'a' 'a']\
    TEST GROUP: ['b' 'b' 'b' 'b' 'b']
    TRAIN: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10] TEST: [11, 12, 13, 14]
    TRAIN GROUP: ['a' 'a' 'a' 'a' 'a' 'a' 'b' 'b' 'b' 'b' 'b']\
    TEST GROUP: ['c' 'c' 'c' 'c']
    TRAIN: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]\
    TEST: [15, 16, 17]
    TRAIN GROUP: ['a' 'a' 'a' 'a' 'a' 'a' 'b' 'b' 'b' 'b' 'b' 'c' 'c' 'c' 'c']\
    TEST GROUP: ['d' 'd' 'd']
    """
    @_deprecate_positional_args
    def __init__(self,
                 n_splits=5,
                 *,
                 max_train_size=None
                 ):
        super().__init__(n_splits, shuffle=False, random_state=None)
        self.max_train_size = max_train_size

    def split(self, X, y=None, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        groups : array-like of shape (n_samples,)
            Group labels for the samples used while splitting the dataset into
            train/test set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        """
        if groups is None:
            raise ValueError(
                "The 'groups' parameter should not be None")
        X, y, groups = indexable(X, y, groups)
        n_samples = _num_samples(X)
        n_splits = self.n_splits
        n_folds = n_splits + 1
        group_dict = {}
        u, ind = np.unique(groups, return_index=True)
        unique_groups = u[np.argsort(ind)]
        n_samples = _num_samples(X)
        n_groups = _num_samples(unique_groups)
        for idx in np.arange(n_samples):
            if (groups[idx] in group_dict):
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds={0} greater than"
                 " the number of groups={1}").format(n_folds,
                                                     n_groups))
        group_test_size = n_groups // n_folds
        group_test_starts = range(n_groups - n_splits * group_test_size,
                                  n_groups, group_test_size)
        for group_test_start in group_test_starts:
            train_array = []
            test_array = []
            for train_group_idx in unique_groups[:group_test_start]:
                train_array_tmp = group_dict[train_group_idx]
                train_array = np.sort(np.unique(
                                      np.concatenate((train_array,
                                                      train_array_tmp)),
                                      axis=None), axis=None)
            train_end = train_array.size
            if self.max_train_size and self.max_train_size < train_end:
                train_array = train_array[train_end -
                                          self.max_train_size:train_end]
            for test_group_idx in unique_groups[group_test_start:
                                                group_test_start +
                                                group_test_size]:
                test_array_tmp = group_dict[test_group_idx]
                test_array = np.sort(np.unique(
                                              np.concatenate((test_array,
                                                              test_array_tmp)),
                                     axis=None), axis=None)
            yield [int(i) for i in train_array], [int(i) for i in test_array]

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras.layers import Reshape, Input, Dense, Activation,Dropout,Flatten,BatchNormalization
from tensorflow.keras.models import Model,Sequential

## Creates a basic double stacked auto encoder to pair with the Temporal Neural Network

In [ ]:
def build_auto_encoder(shape,parameters):
    encoder = Sequential()
    encoder.add(tf.keras.Input((shape,)))
    # encoder.add(tf.keras.layers.BatchNormalization(input_shape = (shape,)))
    # encoder.add(tf.keras.layers.GaussianNoise(0.2))
    encoder.add(Flatten())
    encoder.add(Dense(int(shape*.65),activation = 'relu',input_shape = (shape,)))
    encoder.add(Dense(int(shape*.55),activation = 'relu',input_shape = (shape,)))

    encoder.add(Dense(parameters,activation = 'relu',input_shape = (shape,)))
    
    decoder = Sequential()
    decoder.add(tf.keras.Input(parameters))
    decoder.add(Dense(int(parameters*(100/55)),activation='relu'))
    decoder.add(Dense(int(parameters*(100/55)*(100/65)),activation = 'relu'))
    # decoder.add(tf.keras.layers.Dropout(0.2))
    # decoder.add(tf.keras.layers.GaussianNoise(0.2))
    decoder.add(Dense(shape,activation = 'sigmoid',input_shape = (shape,)))
    decoder.add(Reshape((shape,)))
    return (encoder,decoder)

In [ ]:
class AutoEncoder(Model):
    def __init__(self,orig_shape,latent_dim):
        super(AutoEncoder,self).__init__()
        self.latent_dim = latent_dim
        self.encoder,self.decoder = build_auto_encoder(orig_shape,latent_dim)
    def call(self,x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded
    def get_model():
        return AutoEncoder(name='Auto_Encoder')

In [ ]:
def create_autoencoder(shape,parameters):
    encoder_input = tf.keras.Input((shape,))
    x = tf.keras.layers.Flatten()(encoder_input)
    x = tf.keras.layers.Dense(int(shape*.65),activation = 'relu',input_shape = (shape,))(x)
    x = tf.keras.layers.Dense(int(shape*.55),activation = 'relu',input_shape = (shape,))(x)
    encoder_output  = tf.keras.layers.Dense(parameters,activation = 'relu',input_shape = (shape,))(x)
    

    encoder = tf.keras.Model(encoder_input, encoder_output, name="encoder")
#     encoder.summary()

    x = tf.keras.layers.Reshape((parameters,))(encoder_output)
    x = tf.keras.layers.Dense(int(parameters*(100/55)),activation='relu')(x)
    x = tf.keras.layers.Dense(int(parameters*(100/55)*(100/65)),activation = 'relu')(x)
    x = tf.keras.layers.Dense(shape,activation = 'sigmoid',input_shape = (shape,))(x)
    
    decoder_output = Reshape((shape,))(x)

    autoencoder = tf.keras.Model(encoder_input, decoder_output, name="autoencoder")
#     autoencoder.summary()
    return autoencoder,encoder


In [ ]:
import tensorflow as tf
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss
def auroc(y_true, y_pred):
    
        return tf.py_function(roc_auc_score, (y_true, y_pred), tf.double)

### Creates a Basic Neural Network to compare with the TCN

In [ ]:
def Neural_Network(shape,learning_rate,label_smoothing)->Model:
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Input(shape = shape))
    model.add(Dense(32, activation='relu'))
    # model.add(tf.keras.layers.Dropout(.6))
    model.add(Dense(16, activation='relu'))
    
    model.add(tf.keras.layers.GaussianNoise(0.1))

    model.add(Dense(2,activation = 'softmax',kernel_regularizer='l2'))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = learning_rate),loss = tf.keras.losses.BinaryCrossentropy(label_smoothing = label_smoothing),metrics=auroc)
    return model

## Basic Temproal Convultional Network 

This uses concepts from this paper https://towardsdatascience.com/temporal-convolutional-networks-the-next-revolution-for-time-series-8990af826567 

I experimented with stronger filters and pool sizes as well as more upsampling and transpose layers, I found this simple model to be the most effective.

In [ ]:
def TCN_Model(shape,learning_rate,label_smoothing)->Model:
    model = tf.keras.Sequential()
    
    model.add(tf.keras.layers.Reshape((shape,1),input_shape=(shape,)))
    model.add(tf.keras.layers.Conv1D(filters = 3,kernel_size=2,padding='valid',activation = 'relu'))
    model.add(tf.keras.layers.MaxPooling1D(pool_size=2,strides=1,padding='valid'))
    
#     model.add(tf.keras.layers.SpatialDropout1D(.2))
    model.add(tf.keras.layers.UpSampling1D(size = 2))
    model.add(tf.keras.layers.Conv1DTranspose(filters = 3,kernel_size=2,padding='valid',activation = 'relu'))
#     model.add(tf.keras.layers.SpatialDropout1D(.2))
    model.add(Flatten())
    # model.add(tf.keras.layers.GaussianNoise(0.2))
    # model.add(tf.keras.layers.BatchNormalization())
    model.add(Dense(64, activation='relu'))
#     model.add(Dropout(.2))
    model.add(Dense(2,activation = 'softmax'))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = learning_rate),loss = tf.keras.losses.BinaryCrossentropy(label_smoothing = label_smoothing),metrics=auroc)
    return model

In [ ]:
batch_size = 4096
label_smoothing = 1e-2
learning_rate = 1e-3

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer

In [ ]:
def one_hot(y_true):
    n_y = []
    for val in y_true:
        if val == 1:
            n_y.append([0,1])
        else:
            n_y.append([1,0])
    n_y = np.array(n_y)
    return n_y

In [ ]:
tnn = TCN_Model(shape=64,learning_rate = learning_rate,label_smoothing = label_smoothing)

In [ ]:
tnn.summary()

In [ ]:
nn = Neural_Network(shape=64,learning_rate = learning_rate,label_smoothing = label_smoothing)

In [ ]:
nn.summary()

In [ ]:
from sklearn.metrics import explained_variance_score
from sklearn.decomposition import PCA


In [ ]:
def calculate_tf_positive_rates(y_true,y_pred):
    true_pos = 0
    false_pos = 0
    false_neg = 0
    true_neg = 0
    for true,pred in zip(y_true,y_pred):
        t = np.argmax(true)
        p = np.argmax(pred)
        if t == p:
            if t == 1:
                true_pos+=1
            if t == 0:
                false_pos+=1
        else:
            if p == 1:
                true_neg+=1
            if p == 0:
                false_neg +=1
    
    true_pos_rate = true_pos/(true_pos+false_neg)
    false_pos_rate = false_pos/(false_pos+true_neg)
    return float(true_pos_rate),float(false_pos_rate)
                



In [ ]:
def one_hot(y_vals):
    one_h = []
    for val in y_vals:
        if val == 1:
            one_h.append([0,1])
        else:
            one_h.append([1,0])
    one_h = np.array(one_h)
    return one_h

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
# plt.style.use('ggplot')
color_pal =  [x['color'] for x in plt.rcParams['axes.prop_cycle']]


In [ ]:
def graph_roc_auc_curve(results):
    roc_set = {}
    res = np.array(results)
    roc_set['True Positive'] = res[:,0]
    roc_set['True Negative'] = res[:,1]
    roc_set['ROC/AUC'] = res[:,2]
    roc_set['Model'] = res[:,3]
    roc_set['Dimension reduction'] = res[:,4]
    roc_set['True Positive'] = roc_set['True Positive'].astype('float32',copy=True)
    roc_set['True Negative'] = roc_set['True Negative'].astype('float32',copy=True)
    roc_df = pd.DataFrame.from_dict(roc_set)
    roc_df = roc_df.sort_values(by='True Positive',ascending=True)
    roc_df = roc_df.sort_values(by='True Negative',ascending=True)
    
    # fig, ax = plt.subplots()
    # ax.plot(roc_df['True Positive'],roc_df['True Negative'])
    # plt.show()
    g = sns.lmplot(x='True Negative', y='True Positive', data = roc_df,hue = 'Model',markers=False)
    g.set(ylim=(0, 1.1))
    g.set(xlim=(0, 1.1))
    plt.show()

In [ ]:
def graph_roc_auc_curve(result_table):
    fig = plt.figure(figsize=(15,10))

    for i in result_table.index:
        plt.plot(result_table.loc[i]['fpr'], 
                result_table.loc[i]['tpr'], 
                label="{}, AUC={:.3f}".format(result_table.loc[i]['classifiers'], result_table.loc[i]['auc']))
    
    plt.plot([0,1], [0,1], color='orange', linestyle='--')

    plt.xticks(np.arange(0.0, 1.1, step=0.1))
    plt.xlabel("Flase Positive Rate", fontsize=15)

    plt.yticks(np.arange(0.0, 1.1, step=0.1))
    plt.ylabel("True Positive Rate", fontsize=15)

    plt.title('ROC Curve Analysis', fontweight='bold', fontsize=15)
    plt.legend(prop={'size':13}, loc='lower right')

    plt.show()

In [ ]:
from sklearn.metrics import roc_curve, auc
from typing import Tuple, List, Dict
import time

In [ ]:
def get_best_model(results):
    roc_set = {}
    res = np.array(results)
    roc_set['True Positive'] = res[:,0]
    # roc_set['True Positive'] = roc_set['True Positive'].mean()
    roc_set['True Negative'] = res[:,1]
    # roc_set['True Negative']= roc_set['True Negative'].mean()
    roc_set['ROC/AUC'] = res[:,2]
    roc_set['Model'] = res[:,3]
    # roc_set['ROC/AUC'] = roc_set['ROC/AUC'].mean()
    roc_set['Dimension reduction'] = res[:,4]
    
    roc_set['True Positive'] = roc_set['True Positive'].astype('float32',copy=True)
    roc_set['True Negative'] = roc_set['True Negative'].astype('float32',copy=True)

    roc_df = pd.DataFrame.from_dict(roc_set)
    # return roc_df
    roc_df = roc_df.sort_values(by='True Positive',ascending=True)
    roc_df = roc_df.sort_values(by='True Negative',ascending=True)
    return roc_df[roc_df['ROC/AUC'] == roc_df['ROC/AUC'].max()].values

In [ ]:
def m_roc_curve(y_true,y_pred):
    y_t = []
    y_p = []
    for t in y_true:
        y_t.append(np.argmax(t))
    for p in y_pred:
        y_p.append(np.argmax(p))
    fpr,tpr,_ = roc_curve(np.array(y_t),np.array(y_p))
    return fpr,tpr

In [ ]:
def success_fail_standard(X,Y):
    successes = []
    fails = []
    for features,val in zip(X,Y):
        if val == 1:
            successes.append(features)
        else:
            fails.append(features)
    successes = np.array(successes)
    fails = np.array(fails)
    s_standard = StandardScaler()
    f_standard = StandardScaler()
    s_standard.fit(successes)
    f_standard.fit(fails)
    return s_standard,f_standard

In [ ]:
def standardize_by_fs(X,s_std,f_std):
    s = s_std.transform(X)
    f = f_std.transform(X)
    new_x = []
    for sval,fval in zip(s,f):
        new_x.append(np.append(sval,fval))
    new_x = np.array(new_x)
    return new_x

In [ ]:
import statistics as stat

# Cross validation comparing the TCN with the double stacked auto encoder and the Neural Network. 

In [ ]:
TRAINING = True

In [ ]:
import pickle

Graphs the roc/auc curve at the end of each fold to test performance

In [ ]:
if TRAINING:

    oof = np.zeros(len(train['action']))
    gkf = GroupTimeSeriesSplit(n_splits = 4)
    verbose=  0
    for fold, (tr, te) in enumerate(gkf.split(train['action'].values, train['action'].values, train['date'].values)):
        results_set = []
        Standardizer = StandardScaler()
        Transformer = Normalizer()
        X_tr, X_val = train.loc[tr, features].values, train.loc[te, features].values
        y_tr, y_val = train.loc[tr, 'action'].values, train.loc[te, 'action'].values
        success_standardizer,fail_standardizer = success_fail_standard(X_tr,y_tr)
        len(X_tr[0])
        X_tr = Standardizer.fit_transform(X_tr)
        X_val = Standardizer.transform(X_val)
        with open('Standardizer.pkl', 'wb') as pickle_file:
            pickle.dump(Standardizer, pickle_file)
    # X_val = Transformer.transform(X_val)
        result_table = pd.DataFrame(columns = ['classifiers', 'fpr','tpr','auc'])
        y_tr = one_hot(y_tr)
        y_val = one_hot(y_val)
        shape = len(X_tr[0])
        size = len(X_tr)
#         auto_encoder = AutoEncoder(shape,64)
        auto_encoder, encoder = create_autoencoder(X_tr.shape[-1],64)
        auto_encoder.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = learning_rate),loss = 'mse',metrics = 'mse')
    # shape = int(shape*.75)
        epochs = 1000
        ae_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10,mode = 'min',restore_best_weights = True)
        auto_encoder.fit(x = X_tr,y = X_tr,epochs = 1000,shuffle = False ,batch_size = 4096,verbose = 0,callbacks = [ae_callback],validation_data = (X_val,X_val))
        auto_encoder.evaluate(X_val,X_val)
        X_tr = encoder(X_tr).numpy()
        X_val = encoder(X_val).numpy()
        encoder.save('encoder.h5')

        callback = tf.keras.callbacks.EarlyStopping(monitor='val_auroc',patience = 100,mode = 'max',restore_best_weights = True)
        shape = len(X_tr[0])
        temporal_neural_net = TCN_Model(shape=shape,learning_rate = learning_rate,label_smoothing = label_smoothing)

        print("Train TCN")
        temporal_neural_net.fit(X_tr,y_tr,batch_size = batch_size,epochs=epochs,shuffle = False,verbose = verbose,validation_data=(X_val,y_val),callbacks = [callback])
        neural_network = Neural_Network(shape = shape,learning_rate = learning_rate,label_smoothing = label_smoothing)
        temporal_neural_net.save('tcn-ae.h5')
        print("Train NN")
        
#         print("TCN results are:")
#         temporal_neural_net.evaluate(X_val,y_val)
        tcn_predictions = temporal_neural_net.predict(X_val)
        tcn_true_pos,tcn_false_neg = m_roc_curve(y_val,tcn_predictions)
        result_table = result_table.append({'classifiers':'tcn-ae',
                                            'fpr':tcn_true_pos, 
                                            'tpr':tcn_false_neg, 
                                            'auc':roc_auc_score(y_val,tcn_predictions)}, ignore_index=True)
        results_set.append([np.mean(tcn_true_pos),np.mean(tcn_false_neg),roc_auc_score(y_val,tcn_predictions),'tcn-ae','autoencoder'])
        neural_network.fit(X_tr,y_tr,batch_size = batch_size, epochs=epochs,shuffle = False,verbose = verbose,validation_data = (X_val,y_val),callbacks= [callback])
        
        neural_network.save('nn-ae.h5')
        print("TCN ROC/AUC score is : ", roc_auc_score(y_val,tcn_predictions))
#         print("Neural Network results are:")
#         neural_network.evaluate(X_val,y_val)
    
        nn_predictions = neural_network(X_val,training=False).numpy()
        nn_true_pos,nn_false_neg = m_roc_curve(y_val,nn_predictions)
        result_table = result_table.append({'classifiers':'nn-ae',
                                            'fpr':nn_true_pos, 
                                            'tpr':nn_false_neg, 
                                            'auc':roc_auc_score(y_val,nn_predictions)}, ignore_index=True)
        results_set.append([np.mean(nn_true_pos),np.mean(nn_false_neg),roc_auc_score(y_val,nn_predictions),'nn-ae','autoencoder'])

        print("NN ROC/AUC score is : ", roc_auc_score(y_val,nn_predictions))
      
       
        ensembeled = []
        for v1,v2 in zip(tcn_predictions,nn_predictions):
            p_zero = stat.mean([v1[0],v2[0]])
            p_one = stat.mean([v1[1],v2[1]])
            ensembeled.append([p_zero,p_one])
        ensembeled = np.array(ensembeled)
        ensemble_true_pos,ensemble_false_neg = m_roc_curve(y_val,ensembeled)
        results_set.append([np.mean(ensemble_true_pos),np.mean(ensemble_false_neg),roc_auc_score(y_val,ensembeled),'ensemble','e'])    
        result_table = result_table.append({'classifiers':'ensemble',
                                            'fpr':ensemble_true_pos, 
                                            'tpr':ensemble_false_neg, 
                                            'auc':roc_auc_score(y_val,ensembeled)}, ignore_index=True)
        print("Ensemble ROC/AUC scores are:",roc_auc_score(y_val,ensembeled))
        print(f"train split shape is {X_tr.shape}, validation split shape is {X_val.shape}")
        best_model = get_best_model(results_set)
        print("------------Best Performing Model is --------------")
        print(best_model[0][3])
        print(best_model[0][3],'ROC/AUC is', best_model[0][2])
        print(f"train split shape is {X_tr.shape}, validation split shape is {X_val.shape}")
        train_steps = int(len(tr)//batch_size)

        val_steps = int(len(te)//batch_size)
        graph_roc_auc_curve(result_table)
        print(f"batch_size is {batch_size}")
        print(f"train_steps is {train_steps}, val_steps is {val_steps}")


In [ ]:
models = [temporal_neural_net]

In [ ]:
TRAINING = False

## Submission

In [ ]:
from tqdm import tqdm
from random import choices
import random


import kerastuner as kt


In [ ]:
TRAINING = False

In [ ]:
def load_model(name:str):
    return tf.keras.models.load_model(name)

In [ ]:
checking = load_model('./encoder.h5')

In [ ]:
print(tf.__version__)



In [ ]:
import janestreet

In [ ]:
if not  TRAINING:
    f = np.median
    models = models[-2:]
    import janestreet
    env = janestreet.make_env()
    th = 0.5
    for (test_df, pred_df) in tqdm(env.iter_test()):
        if test_df['weight'].item() > 0:
            x_tt = test_df.loc[:, features].values
            if np.isnan(x_tt[:, 1:].sum()):
                x_tt[:, 1:] = np.nan_to_num(x_tt[:, 1:]) + np.isnan(x_tt[:, 1:]) * f_mean
            x_tt = Standardizer.transform(x_tt)
            x_tt = encoder(x_tt,training=False).numpy()
            pred = np.mean([model(x_tt, training = False).numpy() for model in models],axis=0)
            pred = f(pred)
            pred_df.action = np.where(pred >= th, 1, 0).astype(int)
        else:
            pred_df.action = 0
        env.predict(pred_df)

In [ ]:
# if not TRAINING:
#     import janestreet
#     #janestreet.competition.make_env.__called__ = False
#     env = janestreet.make_env()
#     th = 0.5
#     #w = np.asarray([0.1,0.1,0.1,0.5,0.2])
    
#     for (test_df, pred_df) in tqdm(env.iter_test()):
#         if test_df['weight'].item() > 0:
#             x_tt = test_df.loc[:, features]
#             if np.isnan(x_tt.iloc[:, 1:].values.sum()):
                
#                 x_tt.iloc[:, 1:] = np.nan_to_num(x_tt.iloc[:, 1:]) + np.isnan(x_tt.iloc[:, 1:]) * f_mean

            
#             x_tt = x_tt.values
#             x_tt = Standardizer.transform(x_tt)
#             x_tt = encoder(x_tt).numpy()
#             print(temporal_neural_net(x_tt,training=False))
#             print(model(x_tt,training=False) for model in models)
#             print(np.mean([model(x_tt,training=False) for model in models],axis=0).squeeze())
            
#             pred = np.mean([model(x_tt,training=False) for model in models],axis=0).squeeze()
#             pred = np.mean(pred)
#             pred_df.action = np.where(pred > th, 1, 0).astype(int)
#             print(np.where(pred > th, 1, 0).astype(int))
#         else:
#             pred_df.action = 0
#         env.predict(pred_df)

In [ ]:
pred_df.to_csv('predcitions',index=False)